In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

data = pd.DataFrame({
    "source": ["A", "A", "B", "B", "C", "C", "D", "E", "F", "F", "G", "H", "H", "I", "J"],
    "target": ["B", "C", "C", "D", "A", "E", "C", "F", "C", "G", "C", "I", "C", "J", "C"],
    "weight": [1, 2, 1, 3, 1, 1, 2, 1, 4, 1, 1, 1, 2, 1, 1]
})

G = nx.from_pandas_edgelist(
    data,
    source="source",
    target="target",
    edge_attr="weight",
    create_using=nx.DiGraph
)

dangling_nodes = [n for n in G.nodes if G.out_degree(n) == 0]

pr_global = nx.pagerank(G, alpha=0.85, weight="weight")
print("Global weighted PageRank (top 5):")
for k, v in sorted(pr_global.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {k}: {v:.4f}")

topic_nodes = {"C", "F"}  # pretend these represent a specific topic
personalization = {n: (1.0 / len(topic_nodes) if n in topic_nodes else 0.0) for n in G.nodes}
dangling = personalization.copy()

pr_topic = nx.pagerank(
    G,
    alpha=0.85,
    weight="weight",
    personalization=personalization,
    dangling=dangling,
    max_iter=200,
    tol=1e-08
)

print("\nTopic-biased PageRank toward {C, F} (top 5):")
for k, v in sorted(pr_topic.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {k}: {v:.4f}")

print("\nΔ (topic - global) for nodes (top shifts):")
delta = {n: pr_topic[n] - pr_global[n] for n in G.nodes}
for k, v in sorted(delta.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {k}: {v:+.4f}")

plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42, k=0.6)
node_sizes = [4000 * pr_topic[n] for n in G.nodes]  # scale for visibility
nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes)
nx.draw_networkx_labels(G, pos, font_size=10)
plt.title("Topic-biased PageRank (sizes ∝ PR)")
plt.axis("off")
plt.show()


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.11 install --upgrade pip


ModuleNotFoundError: No module named '_bz2'